# Displaying Scenes

Before we start to constrain and fit the model this is a useful time to learn how to display a scarlet model. [Lupton et al. 2004](https://iopscience.iop.org/article/10.1086/382245) showed the advantage of using the `arcsinh` function to display astrophysical images with a proper normalization to preserve colors while showing objects with a wide range of fluxes. A slightly modified version of their algorithm

$$f(x) = \frac{1}{Q} \sinh^{-1} \left( Q \frac{x-x_\textrm{min}}{\textrm{stretch}} \right)$$

where `Q` is the same as the $\beta$ softening parameter from [Lupton et al. 2004](https://iopscience.iop.org/article/10.1086/382245) and `stretch` determines the size of the linear region, has been implemented in the [LSST software stack](https://github.com/lsst/afw/blob/master/python/lsst/afw/display/rgb/rgbContinued.py#L282-L321) and also in [astropy.visualization.make_lupton_rgb](http://docs.astropy.org/en/stable/api/astropy.visualization.make_lupton_rgb.html#astropy.visualization.make_lupton_rgb). In this document we make use of the image mapping classes contained in the [lupton_rgb.py](https://github.com/astropy/astropy/blob/master/astropy/visualization/lupton_rgb.py) module in astropy to have a bit more control over how we display our images.

*scarlet* adds the additional functionality of mapping multiband images with *more* than three colors into RGB channels. The [img_to_channels](display.ipynb#scarlet.display.img_to_channels) method takes a multiband image `img` as an input and maps all of the bands to RGB colors using the optional `filter_weights` parameter. For example, one band is mapped to all three channels, making a black and white image, while three bands are mapped directly to the RGB channels in reverse order:

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# use a better colormap and don't interpolate the pixels
matplotlib.rc('image', cmap='inferno')
matplotlib.rc('image', interpolation='none')

# Load the sample images
import numpy as np
import scarlet

data = np.load("../../data/hsc_cosmos_35.npz")
images = data["images"]
catalog = data["catalog"]
filters = data["filters"]
weights = 1/data["variance"]
psfs = scarlet.PSF(data["psfs"])

observation = scarlet.Observation(
    images, 
    psf=psfs, 
    weights=weights, 
    channels=filters)

import scarlet.display
from astropy.visualization.lupton_rgb import AsinhMapping, LinearMapping

print("Mapping from 1 band to RGB:\n", scarlet.display.get_default_filter_weight(1))
print("Mapping from 3 bands to RGB:\n", scarlet.display.get_default_filter_weight(3))

In the typical case, where there are more than 3 bands in an image cube, we see that multiple bands are mapped to the same channel, and similarly a single band may be mapped to multiple channels:

In [ ]:
print("Mapping from 5 bands to RGB:\n", scarlet.display.get_default_filter_weight(5))

This makes it possible to map the *gri* bands in our image to the RGB colors using the Lupton color scaling using

In [ ]:
# Set the arcsinh color scaling object
asinh = AsinhMapping(minimum=images.min(), stretch=0.1, Q=10)
# Scale the RGB channels for the image
img_rgb = scarlet.display.img_to_rgb(images[:3], norm=asinh)

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
# Mark each source from the catalog in the image
for k, src in enumerate(catalog):
    plt.text(src["x"], src["y"], str(k), color="w")
plt.show()

This can be a useful visualization to see the color gradients that exist between different bands but we want to make sure that our deblender outputs do not produce residuals in any band. For this we can pass the entire `images` array to [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb), using the same `asinh` normalization, and we get a representation of the entire *grizy* image using the 5-band filter weights given above:

In [ ]:
img_rgb = scarlet.display.img_to_rgb(images, norm=asinh)
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

Of course we might want to use a different mapping of bands to RGB, for example mapping $g \rightarrow B$ and $y \rightarrow R$, so we can define our own filter_weights and pass those to [img_to_rgb](display.ipynb#scarlet.display.img_to_rgb):

In [ ]:
filter_weights = scarlet.display.get_default_filter_weight(5)
filter_weights[0, :] = [0, 0, 0, 0, 1]
filter_weights[2, :] = [1, 0, 0, 0, 0]
filter_weights /= filter_weights.sum(axis=1)[:,None]
print("New filer weights:\n", filter_weights)

img_rgb = scarlet.display.img_to_rgb(images, norm=asinh, filter_weights=filter_weights)
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

The code below shows an example of how to display all of the sources in a model to compare it with the same footprint in the observed images and will be used throughout the remainder of this document. It is beyond the scope of this document to explain in detail how this function works with [matplotlib](https://matplotlib.org) to create the plots, but it is a convenience tool to display all sources and components in a given model:

In [ ]:
# Display the sources
def display_sources(sources, observation, norm=None, subset=None, combine=False, show_sed=True):
    """Display the data and model for all sources in a blend
    
    This convenience function is used to display all (or a subset) of
    the sources and (optionally) their SED's.
    """
    if subset is None:
        # Show all sources in the blend
        subset = range(len(sources))
    for m in subset:
        # Load the model for the source
        src = sources[m]
        if hasattr(src, "components"):
            components = len(src.components)
        else:
            components = 1
        # Convolve the model with the psfs in the observation
        model = observation.render(src.get_model())
        # Extract the bounding box that contains the non-zero
        # pixels in the model
        bbox = scarlet.bbox.trim(np.sum(model, axis=0), min_value=1e-2)
        bb = (slice(None), *bbox.slices)
        # Adjust the stretch based on the maximum flux in the model for the current source
        if model.max() > 10 * bg_rms.max():
            norm = AsinhMapping(minimum=model.min(), stretch=model.max()*.05, Q=10)
        else:
            norm = LinearMapping(minimum=model.min(), maximum=model.max())
        
        # Select the image patch the overlaps with the source and convert it to an RGB image
        img_rgb = scarlet.display.img_to_rgb(images[bb], norm=norm)

        # Build a model for each component in the model
        if hasattr(src, "components"):
            rgb = []
            for component in src.components:
                # Convert the model to an RGB image
                _model = observation.render(component.get_model())
                _rgb = scarlet.display.img_to_rgb(_model[bb], norm=norm)
                rgb.append(_rgb)
        else:
            # There is only a single component
            rgb = [scarlet.display.img_to_rgb(model[bb], norm=norm)]

        # Display the image and model
        figsize = [6,3]
        columns = 2
        # Calculate the number of columns needed and shape of the figure
        if show_sed:
            figsize[0] += 3
            columns += 1
        if not combine:
            figsize[0] += 3*(components-1)
            columns += components-1
        # Build the figure
        fig = plt.figure(figsize=figsize)
        ax = [fig.add_subplot(1,columns,n+1) for n in range(columns)]
        ax[0].imshow(img_rgb)
        ax[0].set_title("Data: Source {0}".format(m))
        for n, _rgb in enumerate(rgb):
            ax[n+1].imshow(_rgb)
            if combine:
                ax[n+1].set_title("Initial Model")
            else:
                ax[n+1].set_title("Component {0}".format(n))
        if show_sed:
            frame = src.frame
            if components > 1:
                for comp in src:
                    ax[-1].plot(comp.sed)
            else:
                ax[-1].plot(src.sed)
            ax[-1].set_xticks(range(frame.C))
            ax[-1].set_xticklabels(frame.channels)
            ax[-1].set_title("SED")
            ax[-1].set_xlabel("Band")
            ax[-1].set_ylabel("Intensity")
        # Mark the current source in the image
        y,x = src.pixel_center
        ax[0].plot(x-bb[2].start, y-bb[1].start, 'rx', mew=2)
        plt.tight_layout()
        plt.show()

We can now use this function to display all detected sources in the blend:

In [ ]:
import pickle

fp = open("../hsc_cosmos_35.sca", "rb")
sources = pickle.load(fp)
fp.close()

model_frame = sources[0].frame
observation = observation.match(model_frame)
display_sources(sources, images, norm=asinh)